In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Basic EDA**

**Importing libaries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

**Loading the data**

In [ ]:
nf = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
nf.head()

**Cleaning the Data**

In [ ]:
nf.info()

In [ ]:
#no duplicated entries
nf.duplicated().sum()

In [ ]:
#convertinf date added to date time
#note that there are 10 entries with missing date_added
nf['date_added'] = pd.to_datetime(nf['date_added'])
nf.loc[nf['date_added'].isnull(),:]

In [ ]:
#adding year added and month added column
nf['year_added'] = pd.to_datetime(nf['date_added']).dt.year.astype('Int64')
nf['month_added'] = pd.to_datetime(nf['date_added']).dt.month.astype('Int64')
nf.head()

# Exploratory Data Analysis

**No. of Movies vs TV Shows**

In [ ]:
nf['type'].value_counts().plot(kind='bar', width=0.4, color=['black','red'])

plt.title('No. of Movies and TV Shows')
plt.xticks(rotation=0)

plt.show()

From this graph, it shows that Netflix offers more movie choices as compater to TV shows

**No. of Movies vs TV Shows by release year**

In [ ]:
nf_added_year = nf.groupby('year_added')['type'].value_counts().sort_index().unstack().reset_index()
nf_added_year.head()

In [ ]:
year = nf_added_year['year_added']
year_movie = nf_added_year['Movie']
year_tv = nf_added_year['TV Show']

x_indexes = np.arange(len(year))
width = 0.25

plt.bar(x_indexes-width/2, year_movie, width=width, label='Movie')
plt.bar(x_indexes+width/2, year_tv, width=width, label='TV Show')

plt.legend(loc='upper left')
plt.title('Movies & TV Shows Added per Year')
plt.xticks(ticks=x_indexes, labels=year, rotation=45)
plt.xlabel('Year')

plt.show()

From this graph, it is quite apparent that Netflix has been increasing its movies and TV shows offering since 2016.

Notably, the more a lot more movies has been released over the past years as comparet to TV shows.

Point to note: 2021 figures may not be complete 2021 has not yet ended

**Let's look at Ratings!**

In [ ]:
nf_movies = nf.loc[nf['type'] == 'Movie', :]
nf_tv = nf.loc[nf['type'] == 'TV Show', :]

In [ ]:
nf_movies.groupby('rating').size().sort_values(ascending=False).plot(kind='bar')

plt.title('Movie Ratings')

plt.xlabel('Ratings')
plt.xticks(rotation=45)

plt.show()

In [ ]:
nf_tv.groupby('rating').size().sort_values(ascending=False).plot(kind='bar')

plt.title('TV Ratings')

plt.xlabel('Ratings')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

From the 2 plots above, it seems that the movies and TV shows are targeted towards the more matured audience

**Let's look at Duration!**

In [ ]:
duration_min = pd.DataFrame(nf_movies['duration'].str.replace(' min', ''))
nf_movies = pd.merge(nf_movies, duration_min, left_index=True, right_index=True).rename(columns={'duration_x':'duration', 'duration_y':'duration_min'})
nf_movies['duration_min'] = nf_movies['duration_min'].astype('int64')
nf_movies.reset_index(inplace=True)
nf_movies.drop('index', axis='columns', inplace=True)

In [ ]:
sns.histplot(nf_movies['duration_min'], kde=True)

plt.title('Movie Duration Distribution')

plt.xlabel('Movie Duration (mins)')
plt.ylabel('')

plt.show()

Most movies on Netflix are typically around 1 to 2 hours long

In [ ]:
#Long Movie > 120 mins
#Ave Movie between 60 to 120
#Short Movie below 60

movie_length = []

for duration in nf_movies['duration_min'] :
    if duration > 120:
        movie_length.append('Long Movie')
    elif duration < 60:
        movie_length.append('Short Movie')
    else:
        movie_length.append('Average Movie')

nf_movies['movie_length'] = pd.Series(movie_length)

nf_movies['movie_length'].value_counts().sort_index().plot(kind='bar')

plt.title('Movie Length')

plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

If we group the movie duration according:
* Long Movies Length are more than 2 Hours,
* Average Movies Length are between 1 to 2 Hours, and
* Short Movies Lenth are below 1 Hour

We can further validate our findings

In [ ]:
nf_movies.loc[nf_movies['movie_length'] == 'Long Movie',['title', 'duration_min']].sort_values('duration_min', ascending=False).head(10)

These are the 10 movies with the longest duration!

In [ ]:
#TV show season
nf_tv['duration'].value_counts().plot(kind='bar')

plt.title('TV Show Seasons')

plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

Coincidental or not, the number of TV shows on Netflix decreases as the seasons increase.

We could probably hypothsize that TV shows with longer seasons as less common and it could be attributed that many TV shows are dropped after the first two seasons

In [ ]:
tv_seasons = nf_tv['duration'].str.split(' ', expand=True).rename(columns={0:'num_seasons', 1:'todrop'}).drop('todrop', axis='columns')
nf_tv = pd.merge(nf_tv, tv_seasons, left_index=True, right_index=True)
nf_tv['num_seasons'] = nf_tv['num_seasons'].astype('int64')
nf_tv[['title', 'num_seasons']].sort_values('num_seasons', ascending=False).head(10)

These are the 10 longest running TV Shows!

**Let's look at Genre!**

In [ ]:
#Movie Genres

nf_movies['genre'] = nf_movies['listed_in'].str.split(', ')

movie_genre = Counter()

for row in nf_movies['genre']:
    movie_genre.update(row)

m_genre = []
m_genre_count = []

for genres in dict(movie_genre):
    m_genre.append(genres)
    m_genre_count.append(dict(movie_genre)[genres])

nf_movie_genre = pd.DataFrame([m_genre, m_genre_count]).T.rename(columns={0:'movie_genre', 1:'count'})

nf_movie_genre = nf_movie_genre.sort_values('count', ascending=False)

In [ ]:
plt.barh(nf_movie_genre['movie_genre'][::-1], nf_movie_genre['count'][::-1])

plt.title('Most Common Movie Genres')
plt.xlabel('No. of Movies')

plt.show()

The most common movie genres are namely 'International Movies', 'Dramas' and 'Comedies'

In [ ]:
#Tv Show genre
nf_tv['genre'] = nf_tv['listed_in'].str.split(', ')

tv_genre = Counter()

for row in nf_tv['genre']:
    tv_genre.update(row)

t_genre = []
t_genre_count = []

for genres in dict(tv_genre):
    t_genre.append(genres)
    t_genre_count.append(dict(tv_genre)[genres])
    
nf_tv_genre = pd.DataFrame([t_genre, t_genre_count]).T.rename(columns={0:'tv_genre', 1:'count'})

nf_tv_genre = nf_tv_genre.sort_values('count', ascending=False)

In [ ]:
plt.barh(nf_movie_genre['movie_genre'][::-1], nf_movie_genre['count'][::-1])

plt.title('Most Common Movie Genres')
plt.xlabel('No. of Movies')

plt.show()

Similarly, the most common movie genres are namely 'International Movies', 'Dramas' and 'Comedies'

**Let's look at Country of Productions!**

In [ ]:
#Movie Production Country
nf_movies['country'].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')

plt.title('Movie Production Country')
plt.xlabel('Country')

plt.show()

In terms of movie production, USA, India and UK ranks among the top 3

In [ ]:
#TV Show Production Country
nf_tv['country'].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')

plt.title('TV Show Production Country')
plt.xlabel('Country')

plt.show()

In terms of TV production, USA, UK and Japan ranks among the top 3